In [1]:
import os
import numpy as np
import xml.etree.ElementTree as ET
from collections import OrderedDict
import matplotlib.pyplot as plt
import pandas as pd
import glob
import io
from collections import namedtuple
from PIL import Image
import tensorflow as tf
import tensorflow_datasets as tfds
from tqdm import tqdm

### Set FilePath

In [2]:
LABEL_PATH = r"F:\Minor Project\Monument Object Detection\helper\Report\all_there_is.csv"
ignore_list = ['kotilingeshvara', 'til mahadev narayan temple', 'king statue']
mistake_list = ['degutale temple', 'kritipur tower']
correct_list = ['degu tale temple_KDS', 'kirtipur tower', 'degu tale']

In [3]:
POSSIBLE_LABELS = dict()

In [4]:
def csv_to_label_map_dict(path):
    df = pd.read_csv(path)
    class_list = list(df['class'])
    final_class_list = list()
    for class_name in class_list:
        if class_name not in ignore_list and class_name not in correct_list:
            if class_name in mistake_list:
                if class_name == 'degutale temple' or class_name == 'degu tale':
                    class_name = correct_list[0]
                elif class_name == 'kritipur tower':
                    class_name = correct_list[1]
            final_class_list.append(class_name)
    class_index = 1
    res_dict = {}
    for class_label in final_class_list:
        res_dict[class_label] = class_index
        class_index += 1
    return res_dict

In [5]:
csv_to_label_map_dict(LABEL_PATH)

{'badrinath temple': 1,
 'basantapur tower': 2,
 'bhagavati temple': 3,
 'bhairavnath temple': 4,
 'bhaktapur tower': 5,
 'bhimeleshvara': 6,
 'bhimsen temple': 7,
 'bhupatindra malla column': 8,
 'bhuvana lakshmeshvara': 9,
 'chasin dega': 10,
 'chayasilin mandap': 11,
 'dattatreya temple': 12,
 'degu tale temple_KDS': 13,
 'fasidega temple': 14,
 'gaddi durbar': 15,
 'garud': 16,
 'golden gate': 17,
 'gopinath krishna temple': 18,
 'hanuman idol': 19,
 'indrapura': 20,
 'jagannatha temple': 21,
 'kala-bhairava': 22,
 'kasthamandap': 23,
 'kavindrapura sattal': 24,
 'kedamatha tirtha': 25,
 'kirtipur tower': 26,
 'kumari ghar': 27,
 'lalitpur tower': 28,
 'mahadev temple': 29,
 'narayan temple': 30,
 'national gallery': 31,
 'nyatapola temple': 32,
 'palace of the 55 windows': 33,
 'panchamukhi hanuman': 34,
 'pratap malla column': 35,
 'shiva temple': 36,
 'shveta bhairava': 37,
 'siddhi lakshmi temple': 38,
 'simha sattal': 39,
 'taleju bell_BDS': 40,
 'taleju bell_KDS': 41,
 'talej

In [6]:
POSSIBLE_LABELS = csv_to_label_map_dict(LABEL_PATH)

In [7]:
def check_class_validity(class_name):
    if class_name == 'degutale temple' or class_name == 'degu tale':
        return 'degu tale temple_KDS'
    elif class_name == 'kritipur tower':
        return 'kirtipur tower'
    elif class_name not in list(POSSIBLE_LABELS.keys()):
        return None
    else:
        return class_name

In [8]:
class_list = []
def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        all_objects = root.findall('object')
        changed_filename = xml_file.split('\\')[-1].split('.')[0] + '.JPG'
        # Handling xml parser for random background images
        if len(all_objects) != 0:
            for member in all_objects:
                class_label = check_class_validity(member[0].text)
                if class_label is not None:
                    value = (changed_filename,
                             int(root.find('size')[0].text),
                             int(root.find('size')[1].text),
                             class_label,
                             int(member[4][0].text),
                             int(member[4][1].text),
                             int(member[4][2].text),
                             int(member[4][3].text)
                             )
                    if class_label not in class_list:
                        class_list.append(class_label)
                    xml_list.append(value)
        else:
            # Setting background class to 'bg', and four bbox coordinates all to neg 1.
            value = (changed_filename,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     'bg',
                      -1,
                      -1,
                      -1,
                      -1)
            xml_list.append(value)
    column_name = ['filename', 'width', 'height',
                   'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df


In [9]:
# xml_to_csv(r"F:\Minor Data Collection\Final Image Data\Monument Original\Merged_Original_Augmented\test\Annotations")

In [10]:
def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]

In [11]:
def int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))
def bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))
def float_list_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))
def bytes_list_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=value))
def int64_list_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

In [12]:
def class_text_to_int(row_label):
    return POSSIBLE_LABELS[row_label]

In [13]:
def create_tf_example(group, path):
    with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []
    
    for index, row in tqdm(group.object.iterrows()):
        if int(row['xmin']) > -1:
            xmins.append(row['xmin'] / width)
            xmaxs.append(row['xmax'] / width)
            ymins.append(row['ymin'] / height)
            ymaxs.append(row['ymax'] / height)
            classes_text.append(row['class'].encode('utf8'))
            classes.append(class_text_to_int(row['class']))
            
        else:
            xmins.append(0)
            xmaxs.append(0)
            ymins.append(0)
            ymaxs.append(0)
            classes_text.append(row['class'].encode('utf8'))
            classes.append(0)
    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': int64_feature(height),
        'image/width': int64_feature(width),
        'image/filename': bytes_feature(filename),
        'image/encoded': bytes_feature(encoded_jpg),
        'image/object/bbox/xmin': float_list_feature(xmins),
        'image/object/bbox/xmax': float_list_feature(xmaxs),
        'image/object/bbox/ymin': float_list_feature(ymins),
        'image/object/bbox/ymax': float_list_feature(ymaxs),
        'image/object/class/text': bytes_list_feature(classes_text),
        'image/object/class/label': int64_list_feature(classes),
    }))
    return tf_example

In [16]:
def TFRecord_Creator(image_dir, annotations_dir, output_file, split_type = "BDG"):
    writer = tf.io.TFRecordWriter(output_file)
    path = os.path.join(image_dir)
    examples = xml_to_csv(annotations_dir)
    #grouped by the filename so that multiple monument detection boxes are grouped in a single 'GROUP'
    grouped = split(examples, 'filename') 
    for group in grouped:
        tf_example = create_tf_example(group, path)
        writer.write(tf_example.SerializeToString())
    writer.close()
    print('Successfully created the TFRecord file for ', split_type)

In [17]:
BASE_DIR = "F:\Minor Data Collection\Final Image Data\Monument Original\Merged_Original_Augmented"
annotations_path = 'Annotations'
jpg_images_path = 'JPEGImages'
train_dir = os.path.join(BASE_DIR, 'train')
validation_dir = os.path.join(BASE_DIR, 'validation')
test_dir = os.path.join(BASE_DIR, 'test')

In [20]:
TFRecord_Creator(os.path.join(train_dir, jpg_images_path), os.path.join(train_dir, annotations_path), os.path.join(BASE_DIR, 'train_aug.tfrecord'))

TFRecord_Creator(os.path.join(validation_dir, jpg_images_path), os.path.join(validation_dir, annotations_path), os.path.join(BASE_DIR, 'validation_aug.tfrecord'))

2it [00:00, 2006.84it/s]
1it [00:00, 1001.51it/s]
1it [00:00, 978.15it/s]
1it [00:00, 1002.46it/s]
1it [00:00, ?it/s]
1it [00:00, 1002.22it/s]
1it [00:00, 1015.08it/s]
1it [00:00, 500.33it/s]
1it [00:00, 1045.44it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
3it [00:00, ?it/s]
3it [00:00, ?it/s]
3it [00:00, 3116.90it/s]
3it [00:00, 2920.82it/s]
3it [00:00, 3004.52it/s]
2it [00:00, 982.62it/s]
2it [00:00, 1953.56it/s]
3it [00:00, ?it/s]
3it [00:00, 3010.27it/s]
3it [00:00, 3007.39it/s]
3it [00:00, 3033.49it/s]
3it [00:00, 3005.95it/s]
2it [00:00, ?it/s]
2it [00:00, 1999.19it/s]
2it [00:00, ?it/s]
2it [00:00, 957.60it/s]
2it [00:00, ?it/s]
2it [00:00, 1986.41it/s]
2it [00:00, 2004.45it/s]
2it [00:00, 2003.49it/s]
2it [00:00, 2024.28it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, 1003.66it/s]
1it [00:00, ?it/s]
1it [00:00, 1001.74it/s]
1it [00:00, 1009.95it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
2it [00:00, 2128.01it/s]
2it [00:00, ?it/s]
2it [00:00, 2004.93it/s]
2it [00:00, ?it/s]


3it [00:00, 2900.63it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, 964.21it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
3it [00:00, 3010.27it/s]
3it [00:00, ?it/s]
3it [00:00, ?it/s]
3it [00:00, 3009.55it/s]
3it [00:00, 3157.57it/s]
3it [00:00, ?it/s]
3it [00:00, 2984.56it/s]
3it [00:00, 3008.83it/s]
3it [00:00, 3008.83it/s]
3it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, 1001.51it/s]
1it [00:00, 1002.22it/s]
1it [00:00, 1001.98it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, 1040.00it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, 1001.98it/s]
1it [00:00, 1002.46it/s]
1it [00:00, 976.56it/s]
1it [00:00, ?it/s]
1it [00:00, 1040.25it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
3it [00:00, 3004.52it/s]
3it [00:00, 3135.54it/s]
3it [00:00, 3004.52it/s]
3it [00:00, 3003.80it/s]
3it [00:00, 3005.95it/s]
3it [00:00, 3007.39it/s]
3it [00:00, 2880.70it/s]
1it [00:00, 1091.41it/s]
1it [00:00, 1003.18it/

1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, 1000.31it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, 1003.66it/s]
1it [00:00, 334.31it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, 499.98it/s]
1it [00:00, 1002.46it/s]
1it [00:00, 1000.55it/s]
1it [00:00, 1001.74it/s]
1it [00:00, 1004.62it/s]
1it [00:00, 1001.74it/s]
1it [00:00, 1002.46it/s]
2it [00:00, 1954.48it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, 1046.22it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
2it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, 1008.25it/s]
1it [00:00, 1002.70it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
2it [00:00, 2061.59it/s]
2it [00:00, 2005.40it/s]
2it [00:00, ?it/s]
2it [00:00, 2004.93it/s]
2it [00:00, 2040.03it/s]
2it [00:00, 2118.87it/s]
1it [00:00, ?it/s]
1it [00:00, 1004.14it/s]
1it [00:00, ?it/s]
1it [00:00, 1009.95it/s]
1it [00:00, ?it/s]
2i

4it [00:00, 2004.45it/s]
4it [00:00, 4000.29it/s]
1it [00:00, ?it/s]
1it [00:00, 990.39it/s]
3it [00:00, 3007.39it/s]
2it [00:00, 2091.40it/s]
2it [00:00, ?it/s]
2it [00:00, 998.05it/s]
2it [00:00, 1002.46it/s]
4it [00:00, 1336.83it/s]
4it [00:00, 2007.08it/s]
4it [00:00, 4014.65it/s]
4it [00:00, 2010.45it/s]
4it [00:00, 4008.89it/s]
4it [00:00, 4011.77it/s]
4it [00:00, 4010.81it/s]
1it [00:00, 1011.89it/s]
1it [00:00, 1005.59it/s]
1it [00:00, 1004.14it/s]
1it [00:00, 1002.94it/s]
2it [00:00, 1002.46it/s]
5it [00:00, 5014.71it/s]
5it [00:00, 1671.57it/s]
5it [00:00, 5013.51it/s]
4it [00:00, 1335.98it/s]
4it [00:00, 4007.94it/s]
4it [00:00, 4010.81it/s]
2it [00:00, ?it/s]
2it [00:00, 2005.40it/s]
2it [00:00, 998.76it/s]
3it [00:00, 1002.62it/s]
3it [00:00, 1503.33it/s]
3it [00:00, 3010.27it/s]
1it [00:00, 1002.70it/s]
1it [00:00, 1000.79it/s]
1it [00:00, 1001.03it/s]
1it [00:00, 1001.03it/s]
1it [00:00, 1002.22it/s]
1it [00:00, 997.22it/s]
3it [00:00, 3007.39it/s]
3it [00:00, 3007.39it/

1it [00:00, 1002.70it/s]
1it [00:00, 966.43it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, 500.10it/s]
1it [00:00, 967.10it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
2it [00:00, 2007.32it/s]
2it [00:00, 1927.09it/s]
2it [00:00, ?it/s]
5it [00:00, 5202.56it/s]
5it [00:00, ?it/s]
5it [00:00, 5006.33it/s]
5it [00:00, ?it/s]
5it [00:00, 4780.38it/s]
2it [00:00, 1943.61it/s]
5it [00:00, ?it/s]
5it [00:00, ?it/s]
2it [00:00, ?it/s]
1it [00:00, 1002.94it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
4it [00:00, 4012.73it/s]
4it [00:00, 1983.83it/s]
4it [00:00, 3868.39it/s]
4it [00:00, 4010.81it/s]
4it [00:00, 4007.94it/s]
4it [00:00, 3842.70it/s]
4it [00:00, ?it/s]
4it [00:00, ?it/s]
1it [00:00, 1001.98it/s]
4it [00:00, 3999.34it/s]
6it [00:00, 6020.53it/s]
6it [00:00, ?it/s]
6it [00:00, 6037.87it/s]
6it [00:00, 3008.11it/s]
6it [00:00, 3007.39it/s]
2it [00:00, 2004.93it/s]
3it [00:00, ?it/s]
1it [00:00, 1002.70it/s]
1it [00:00, ?it/s]
1it [00:00, 1005.11it/s]
1it [00:00,

...

3it [00:00, 3003.80it/s]
3it [00:00, 3008.83it/s]
2it [00:00, 2006.84it/s]
2it [00:00, 2003.97it/s]
2it [00:00, 2007.80it/s]
2it [00:00, 2011.17it/s]
2it [00:00, 1992.54it/s]
2it [00:00, 2005.40it/s]
2it [00:00, 2004.45it/s]
2it [00:00, 1425.18it/s]
2it [00:00, 2003.49it/s]
2it [00:00, 2005.88it/s]
2it [00:00, 2003.97it/s]
2it [00:00, 2005.88it/s]
2it [00:00, 1009.22it/s]
2it [00:00, 596.16it/s]
2it [00:00, 2031.14it/s]
3it [00:00, 1503.33it/s]
1it [00:00, 1003.42it/s]
1it [00:00, 1002.70it/s]
1it [00:00, ?it/s]
1it [00:00, 1945.41it/s]
1it [00:00, 1011.41it/s]
2it [00:00, 2006.84it/s]
2it [00:00, 2003.49it/s]
4it [00:00, 4023.31it/s]
2it [00:00, ?it/s]
2it [00:00, 2010.21it/s]
2it [00:00, 2012.14it/s]
2it [00:00, 2004.45it/s]
3it [00:00, 3016.04it/s]
2it [00:00, 2001.10it/s]
1it [00:00, 501.11it/s]
2it [00:00, 1000.43it/s]
1it [00:00, ?it/s]
3it [00:00, 5264.82it/s]
1it [00:00, 1004.62it/s]
1it [00:00, 1005.83it/s]
1it [00:00, 1400.90it/s]
1it [00:00, ?it/s]
1it [00:00, 1002.70it/s]
1

1it [00:00, 1002.70it/s]
2it [00:00, 3462.08it/s]
2it [00:00, 2005.40it/s]
2it [00:00, 2006.36it/s]
2it [00:00, 2005.88it/s]
1it [00:00, 1003.18it/s]
1it [00:00, 1003.42it/s]
1it [00:00, 999.60it/s]
1it [00:00, 1003.66it/s]
3it [00:00, 3359.92it/s]
2it [00:00, ?it/s]
2it [00:00, 2008.29it/s]
3it [00:00, 1001.74it/s]
2it [00:00, 2005.40it/s]
2it [00:00, 2002.53it/s]
2it [00:00, 2004.93it/s]
1it [00:00, 1003.90it/s]
1it [00:00, ?it/s]
2it [00:00, 2007.32it/s]
2it [00:00, 2006.36it/s]
1it [00:00, 1002.94it/s]
2it [00:00, 1001.62it/s]
3it [00:00, 3005.95it/s]
2it [00:00, 1002.34it/s]
2it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, 1002.22it/s]
1it [00:00, 1003.66it/s]
1it [00:00, 1002.70it/s]
1it [00:00, 1002.22it/s]
1it [00:00, 500.87it/s]
1it [00:00, 330.65it/s]
1it [00:00, 502.55it/s]
1it [00:00, 498.97it/s]
3it [00:00, 1507.66it/s]
4it [00:00, 4014.65it/s]
4it [00:00, 2005.17it/s]
4it [00:00, 4013.69it/s]
1it [00:00, 1002.70it/s]
4it [00:00, 4011.77it/s]
4it [00:00, 2136.68it/s]
4it 

2it [00:00, 2002.53it/s]
3it [00:00, 1488.93it/s]
2it [00:00, 2005.88it/s]
2it [00:00, 2038.05it/s]
2it [00:00, 2005.40it/s]
3it [00:00, 1674.37it/s]
4it [00:00, 1001.92it/s]
3it [00:00, 3008.11it/s]
1it [00:00, 111.41it/s]
3it [00:00, 3008.83it/s]
5it [00:00, 1252.93it/s]
3it [00:00, 3013.87it/s]
2it [00:00, 2004.45it/s]
3it [00:00, 2999.50it/s]
2it [00:00, 2004.45it/s]
2it [00:00, 1002.58it/s]
1it [00:00, ?it/s]
2it [00:00, 3015.32it/s]
2it [00:00, 2008.29it/s]
2it [00:00, ?it/s]
2it [00:00, 650.28it/s]
2it [00:00, 3390.71it/s]
2it [00:00, 2006.36it/s]
1it [00:00, 1002.46it/s]
2it [00:00, 2005.88it/s]
1it [00:00, 1002.70it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, 1000.79it/s]
2it [00:00, 2003.97it/s]
1it [00:00, 997.69it/s]
1it [00:00, 1003.18it/s]
1it [00:00, 1141.93it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, 1002.46it/s]
1it [00:00, 1001.27it/s]
1it [00:00, 1002.22it/s]
1it [00:00, ?it/s]
1it [00:00, 500.33it/s]
1it [00:00, ?it/s]
3it [00:00, 3014.59it/s]
1i

1it [00:00, ?it/s]
1it [00:00, 1002.94it/s]
1it [00:00, 1008.00it/s]
1it [00:00, 1002.22it/s]
1it [00:00, ?it/s]
1it [00:00, 1003.18it/s]
3it [00:00, 1503.51it/s]
1it [00:00, 1180.50it/s]
1it [00:00, 1002.22it/s]
1it [00:00, 1003.66it/s]
1it [00:00, 1000.55it/s]
1it [00:00, 1001.74it/s]
1it [00:00, 1001.74it/s]
1it [00:00, 1002.70it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, 1001.03it/s]
1it [00:00, 1002.70it/s]
1it [00:00, 1001.98it/s]
1it [00:00, 996.75it/s]
1it [00:00, 1003.18it/s]
1it [00:00, ?it/s]
1it [00:00, 1002.70it/s]
1it [00:00, 334.07it/s]
1it [00:00, 1001.51it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, 1001.74it/s]
1it [00:00, 1002.46it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, 500.51it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, 1000.55it/s]
3it [00:00, 1505.31it/s]
1it [00:00, 1002.22it/s]
1it [00:00, 1000.55it/s]
1it [00:00, ?it/s]
1it [00:00, 1854.25it/s]
1it [00:00, 1007.76it/s]
1it [00:00, 1114.02it/s]
1it [00:00, 1153.55it/s]


Successfully created the TFRecord file for  BDG


In [74]:
# image_dir = './Dataset/test/JPEGImages'
# annotations_dir = './Dataset/test/Annotations'
# output_file = './monument-test.tfrecord'
# main(image_dir, annotations_dir, output_file)

In [ ]:
# def xml_to_csv(path):
#     xml_list = []
#     for xml_file in glob.glob(path + '/*.xml'):
#         tree = ET.parse(xml_file)
#         root = tree.getroot()
#         for member in root.findall('object'):
#             class_label = check_class_validity(member[0].text)
#             if class_label is not None:
#                 value = (root.find('filename').text,
#                          int(root.find('size')[0].text),
#                          int(root.find('size')[1].text),
#                          class_label,
#                          int(member[4][0].text),
#                          int(member[4][1].text),
#                          int(member[4][2].text),
#                          int(member[4][3].text)
#                          )
#                 xml_list.append(value)
#     column_name = ['filename', 'width', 'height',
#                    'class', 'xmin', 'ymin', 'xmax', 'ymax']
#     xml_df = pd.DataFrame(xml_list, columns=column_name)
#     return xml_df